In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
import string
import re
import pandas as pd
import time
import os
import numpy as np
import pickle

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# DEFINE DIRECTORY PATH
path_to_json = 'TrainingData/'

# CREATE LIST OF FILES FROM THE DIRECTORY
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# DEFINE PANDAS DATAFRAME
df = pd.DataFrame()

# LOOP THROUGH FILES, READ IN JSON AND BUILD DATAFRAME
for index, js in enumerate(json_files):
    json_data = pd.read_json(os.path.join(path_to_json, js), 'r')
    df = df.append(json_data)
    
# LOOK AT TOPIC DICTIONARY AND GET A TOPIC COUNT
topic_file = open('./topicDictionary.txt', 'r')
topics = topic_file.read().split('\r\n')

# SPLITTING THE ELEMENTS OF THE JSON INTO TEXT, PUBLICATION DATE AND TOPICS
df['text'] = df.TrainingData.apply(lambda x: x['bodyText'])
df['pubdate'] = df.TrainingData.apply(lambda x: x['webPublicationDate'])
df['topics'] = df.TrainingData.apply(lambda x: x['topics'])

# DROP FIRST TWO COLUMNS
df.reset_index(inplace=True, drop=True)
df.drop('TrainingData', axis=1, inplace=True)

# DEFINE FUNCTION TO CREATE OUR DATAFRAME
def topic_col(x):
    a = 0
    for elem in x:
        if elem == topic:
            a = 1
    return a

# RUN TOPIC COL FUNCTION ON ALL DATA
for topic in topics:
    time1 = time.time()
    df[topic] = df['topics'].map(topic_col)
    print topic
    time2 = time.time()
    time_in_s = (time2-time1)
    print 'Function takes around %0.3f seconds to run' % (time_in_s)

activism
Function takes around 14.344 seconds to run
afghanistan
Function takes around 3.619 seconds to run
aid
Function takes around 1.084 seconds to run
algerianhostagecrisis
Function takes around 1.309 seconds to run
alqaida
Function takes around 1.219 seconds to run
alshabaab
Function takes around 1.263 seconds to run
antiwar
Function takes around 1.234 seconds to run
arabandmiddleeastprotests
Function takes around 1.447 seconds to run
armstrade
Function takes around 1.173 seconds to run
australianguncontrol
Function takes around 1.262 seconds to run
australiansecurityandcounterterrorism
Function takes around 1.460 seconds to run
bastilledaytruckattack
Function takes around 1.281 seconds to run
belgium
Function takes around 1.261 seconds to run
berlinchristmasmarketattack
Function takes around 1.619 seconds to run
bigdata
Function takes around 1.210 seconds to run
biometrics
Function takes around 1.196 seconds to run
bokoharam
Function takes around 1.214 seconds to run
bostonmarath

undercoverpoliceandpolicing
Function takes around 1.413 seconds to run
unitednations
Function takes around 1.239 seconds to run
usguncontrol
Function takes around 1.240 seconds to run
values
Function takes around 1.097 seconds to run
warcrimes
Function takes around 1.236 seconds to run
warreporting
Function takes around 1.204 seconds to run
weaponstechnology
Function takes around 1.243 seconds to run
womeninbusiness
Function takes around 1.211 seconds to run
woolwichattack
Function takes around 1.274 seconds to run
worldmigration
Function takes around 1.240 seconds to run
zikavirus
Function takes around 1.156 seconds to run


In [3]:
# CREATE FINAL TEST DF
testdf = pd.read_json('/Users/sudheerpamula/Downloads/TestData.json', 'r')
testdf['text'] = testdf.TestData.apply(lambda x: x['bodyText'])
testdf['pubdate'] = testdf.TestData.apply(lambda x: x['webPublicationDate'])
testdf['topics'] = testdf.TestData.apply(lambda x: x['topics'])
testdf.drop('TestData', axis=1, inplace=True)

In [4]:
def sampler(topic, df):
    
    # SELECT INDICES OF TOPIC ARTICLES
    topicindexes = df[df[topic] == 1].index.tolist()
    
    # FIND COUNT OF TOPIC ARTICLES
    articlecount = len(topicindexes)
    
    if articlecount < 100:
        print articlecount
        # SELECT NON TOPIC INDICES
        nontopicarticlesindexes = df[df[topic] == 0].sample(articlecount*9).index.tolist()
        nonarticlecount = len(nontopicarticlesindexes)
        # CREATE LIST OF COMBINED INDICES
        sampleindex = topicindexes + nontopicarticlesindexes 
        # CREATE NEW DATAFRAME
        X = df.iloc[sampleindex]['text'].reset_index(drop=True)
        y = df.iloc[sampleindex][topic].reset_index(drop=True)
        X = X.append(df.iloc[topicindexes]['text'])
        y = y.append(df.iloc[topicindexes][topic])
        X.reset_index(drop=True, inplace=True)
        y.reset_index(drop=True, inplace=True) 
    else:
        # SELECT NON TOPIC INDICES
        topicindexes = df[df[topic] == 1].sample(100).index.tolist()
        
        nontopicarticlesindexes = df[df[topic] == 0].sample(len(topicindexes)*9).index.tolist()
        nonarticlecount = len(nontopicarticlesindexes)
        # CREATE LIST OF COMBINED INDICES
        sampleindex = topicindexes + nontopicarticlesindexes
        # CREATE NEW DATAFRAME
        X = df.iloc[sampleindex]['text'].reset_index(drop=True)
        y = df.iloc[sampleindex][topic].reset_index(drop=True)
    
    return X, y

In [5]:
submission = pd.DataFrame(index=testdf.index, columns=topics)

In [9]:
topics_0 = ['activism', 'turkeycoupattempt', 'tunisiaattack2015', 'sanbernardinoshooting', 'peaceandreconciliation',
            'parisattacks', 'orlandoterrorattack', 'munichshooting', 'francetrainattack', 'charliehebdoattack',
            'brusselsattacks', 'berlinchristmasmarketattack','bastilledaytruckattack','zikavirus']

In [27]:
parameters = {
#     'tfidf__norm': ('l2', 'l1'),
    'clf__penalty': ('l1', 'l2'),
    'clf__C': [0.001, 1, 10, 100]
}

clf = LogisticRegression()

In [17]:
time1 = time.time()

for topic in topics:
    if topic in topics_0:
        submission[topic] = 0
    else:
        X, y = sampler(topic, df)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
        
        #N-GRAM RANGE OF (1,1)
        vectorizer = CountVectorizer(ngram_range=(1,2), lowercase=True, max_df=0.15)

        # PIPELINE OF VECTORIZER, TF-IDF TRANSFORMER, STANDARD SCALER, MODEL
        pipe = Pipeline([('vectorizer', vectorizer), 
                 ('tfidf', TfidfTransformer()),
                 ('ss', StandardScaler(with_mean=False)),
                 ('clf', clf)])

        # GRIDSEARCH PIPELINE FIT
        grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)

        # PREDICT
        preds = grid_search.best_estimator_.predict(X_test)
        topicf1score = f1_score(y_test, preds)
        print("F1 score:", topicf1score)

        # FINAL PREDICTIONS
        preds_final = grid_search.best_estimator_.predict(testdf['text'])
        print grid_search.best_params_

        # WRITE TO CSV
        submission[topic] = preds_final
        
    submission.to_csv('submission_logisticreg_n2_C.csv')
    print topic
    
time2 = time.time()
time_in_s = (time2-time1)
print 'Function takes around %0.3f seconds to run' % (time_in_s)

activism
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.4min finished


('F1 score:', 0.89795918367346939)
{'clf__penalty': 'l1', 'clf__C': 1, 'tfidf__norm': 'l2'}
afghanistan
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.4min finished


('F1 score:', 0.82608695652173914)
{'clf__penalty': 'l1', 'clf__C': 1, 'tfidf__norm': 'l1'}
aid
58
Fitting 3 folds for each of 16 candidates, totalling 48 fits


...

[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.2min finished


('F1 score:', 0.97959183673469385)
{'clf__penalty': 'l1', 'clf__C': 1, 'tfidf__norm': 'l2'}
libya
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.2min finished


('F1 score:', 0.73076923076923084)
{'clf__penalty': 'l1', 'clf__C': 1, 'tfidf__norm': 'l1'}
localgovernment


KeyboardInterrupt: 

In [20]:
time1 = time.time()

for topic in topics:
    if topic in topics_0:
        submission[topic] = 0
    else:
        X, y = sampler(topic, df)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
        
        #N-GRAM RANGE OF (1,1)
        vectorizer = CountVectorizer(ngram_range=(1,2), lowercase=True, max_df=0.15)

        # PIPELINE OF VECTORIZER, TF-IDF TRANSFORMER, STANDARD SCALER, MODEL
        pipe = Pipeline([('vectorizer', vectorizer), 
                 ('tfidf', TfidfTransformer()),
                 ('ss', StandardScaler(with_mean=False)),
                 ('clf', clf)])

        # GRIDSEARCH PIPELINE FIT
        grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)

        # PREDICT
        preds = grid_search.best_estimator_.predict(X_test)
        topicf1score = f1_score(y_test, preds)
        print("F1 score:", topicf1score)

        # FINAL PREDICTIONS
        preds_final = grid_search.best_estimator_.predict(testdf['text'])
        print grid_search.best_params_

        # WRITE TO CSV
        submission[topic] = preds_final
        
    submission.to_csv('submission_logisticreg_n2_C_2.csv')
    print topic
    
time2 = time.time()
time_in_s = (time2-time1)
print 'Function takes around %0.3f seconds to run' % (time_in_s)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.2min finished


('F1 score:', 1.0)
{'clf__penalty': 'l1', 'clf__C': 1, 'tfidf__norm': 'l2'}
logistics
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.3min finished


('F1 score:', 0.57142857142857151)
{'clf__penalty': 'l1', 'clf__C': 10, 'tfidf__norm': 'l1'}
london
Fitting 3 folds for each of 16 candidates, totalling 48 fits


...

('F1 score:', 0.91304347826086962)
{'clf__penalty': 'l1', 'clf__C': 1, 'tfidf__norm': 'l2'}
womeninbusiness
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.0min finished


('F1 score:', 0.87499999999999989)
{'clf__penalty': 'l1', 'clf__C': 1, 'tfidf__norm': 'l1'}
woolwichattack
Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  1.0min finished


('F1 score:', 0.93617021276595747)
{'clf__penalty': 'l1', 'clf__C': 1, 'tfidf__norm': 'l1'}
worldmigration
zikavirus
Function takes around 10091.745 seconds to run
